## 机器学习的一般步骤
首先，我们来看一个经典的神经网络结构，lenet出自论文：Lecun Y, Bottou L, Bengio Y, et al. Gradient-based learning applied to document recognition[J]. Proceedings of the IEEE, 1998, 86(11):2278-2324.
这是一种用于手写体字符识别非常高效的神经网络。
![pic](imgs/nn_lenet.png)
![pic](imgs/nn_lenetb.png)
[卷积神经网络概念](https://cuijiahua.com/blog/2018/01/dl_3.html)

假设我们要用这个网络来对图像进行分类，这项工作一般可以分为以下几个步骤：

1. 问题的抽象：回归问题 还是 分类问题？
2. 数据的准备：整理原始数据，打标签，或者手工标注信息，提取数据作为训练集或测试集
3. 选择适合此问题的模型（神经网络结构）
4. 设置模型、优化器等超参数
5. 数据的预处理（加入随机噪声、随机裁剪、随机旋转 ect.）
6. 训练模型（loss 前向传播 grad 反向传播） 更新模型参数
7. 测试模型（用测试集测试模型的效果）
8. 超参数的优化

其中模型参数的寻优可以说是机器学习的核心之一，其中主要思想是梯度下降及各种优化算法，这里的核心是梯度的反向传播，
如果自己手动编写一个神经网络，需要手工计算大量的梯度反向传播的公式，比较麻烦而且容易推导错误。
计算图(Computation Graph)是现代深度学习框架如PyTorch和TensorFlow等的核心，其为高效自动求导算法——反向传播(Back Propogation)提供了理论支持，了解计算图在实际写程序过程中会有极大的帮助。

总之，使用Pytorch你几乎可以不用关心梯度是怎样反向传播的了。

## pytorch概述
Pytorch是torch的python版本,是由Facebook开源的神经网络框架。配置pytorch在windows和linux下都非常方便。
先安装好python 和 pip 后
进入pytorch官网https://pytorch.org/ 
命令行使用Pip进行安装即可，下面是一个windows下cpu版本的配置。
gpu版本主要是要对cuda进行配置可以较为复杂，这里不详细讲。
```py
pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp37-cp37m-win_amd64.whl
pip3 install torchvision
```
[机器学习常见的七个步骤](https://cloud.tencent.com/developer/news/220760)

### tensor
pytorch的核心是以tensor数据结构展开的。
tensor可以被看作是一个内置了各种操作的**高维数组**，可以使用gpu进行加速运算，而且接口被设计得和**Matlab**与**numpy的ndarrays**很类似。

tensor的类型决定了 其保存的数据的 数据类型：
```
Data type     CPU tensor     GPU tensor
32-bit floating point     torch.FloatTensor     torch.cuda.FloatTensor
64-bit floating point     torch.DoubleTensor     torch.cuda.DoubleTensor
16-bit floating point     torch.HalfTensor     torch.cuda.HalfTensor
8-bit integer (unsigned)     torch.ByteTensor     torch.cuda.ByteTensor
8-bit integer (signed)     torch.CharTensor     torch.cuda.CharTensor
16-bit integer (signed)     torch.ShortTensor     torch.cuda.ShortTensor
32-bit integer (signed)     torch.IntTensor     torch.cuda.IntTensor
64-bit integer (signed)     torch.LongTensor     torch.cuda.LongTensor
```
[这篇文章写了很多tensor的操作](https://blog.csdn.net/zzulp/article/details/80573331 )




In [10]:
import torch as t
x = t.Tensor(5, 3) # 构建 5x3 矩阵，只是分配了空间，未初始化
x = t.Tensor([[1,2],[3,4]])
print(x)

# 在不支持CUDA的机器下，下一步还是在CPU上运行
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
print(device)
x = x.to(device)

tensor([[1., 2.],
        [3., 4.]])
cuda:0


### autograd
深度学习的算法本质上是通过反向传播求导数，而PyTorch的autograd模块则实现了此功能。在Tensor上的所有操作，autograd都能为它们自动提供微分，避免了手动计算导数的复杂过程。
 从0.4起, Variable 正式合并入Tensor, Variable 本来实现的自动微分功能，Tensor就能支持。读者还是可以使用Variable(tensor), 但是这个操作其实什么都没做。建议读者以后直接使用tensor.
 
 PyTorch中`autograd`的底层采用了计算图，计算图是一种特殊的有向无环图（DAG），用于记录算子与变量之间的关系。一般用矩形表示算子，椭圆形表示变量。如表达式$ \textbf {z = wx + b}$可分解为$\textbf{y = wx}$和$\textbf{z = y + b}$，其计算图如图3-3所示，图中`MUL`，`ADD`都是算子，$\textbf{w}$，$\textbf{x}$，$\textbf{b}$即变量。


 ![反向传播](imgs/com_graph.svg)

链式法则：梯度反向传播
如上有向无环图中，$\textbf{X}$和$\textbf{b}$是叶子节点（leaf node），这些节点通常由用户自己创建，不依赖于其他变量。$\textbf{z}$称为根节点，是计算图的最终目标。利用链式法则很容易求得各个叶子节点的梯度。
$${\partial z \over \partial b} = 1,\space {\partial z \over \partial y} = 1\\
{\partial y \over \partial w }= x,{\partial y \over \partial x}= w\\
{\partial z \over \partial x}= {\partial z \over \partial y} {\partial y \over \partial x}=1 * w\\
{\partial z \over \partial w}= {\partial z \over \partial y} {\partial y \over \partial w}=1 * x\\
$$
而有了计算图，上述链式求导即可利用计算图的反向传播自动完成，其过程如图3-4所示。

![计算图的反向传播](imgs/com_graph_backward.svg)


在PyTorch实现中，autograd会随着用户的操作，记录生成当前variable的所有操作，并由此建立一个有向无环图。用户每进行一个操作，相应的计算图就会发生改变。更底层的实现中，图中记录了操作`Function`，每一个变量在图中的位置可通过其`grad_fn`属性在图中的位置推测得到。在反向传播过程中，autograd沿着这个图从当前变量（根节点$\textbf{z}$）溯源，可以利用链式求导法则计算所有叶子节点的梯度。每一个前向传播操作的函数都有与之对应的反向传播函数用来计算输入的各个variable的梯度，这些函数的函数名通常以`Backward`结尾。下面结合代码学习autograd的实现细节。


要想使得Tensor使用autograd功能，只需要设置tensor.requries_grad=True. 
```
# 为tensor设置 requires_grad 标识，代表着需要求导数
# pytorch 会自动调用autograd 记录操作
x = t.ones(2, 2, requires_grad=True)
```
计算下面这个函数的导函数：
$$y = x^3 + 2\times x + 3 + e^x + sin(x)$$
其导数为：
$$ {dy \over dx} = 3 \times x^2 +2 + e^x-cos(x)$$
来看看autograd的计算结果与手动求导计算结果的误差。


In [23]:
import torch as t
def myf(x):
    #计算fx
    y = t.pow(x,3) + 2*x + 3 + t.exp(x) +t.sin(x)
    return y
#手动求导数
def gradmyf(x):
    grad = 3*t.pow(x,2)+ 2 +t.exp(x) +t.cos(x)
    return grad

In [24]:
x = t.randn(3,4,requires_grad = True)
y = myf(x)
print(y)

tensor([[ 2.9593,  2.1927,  4.5930, 13.0516],
        [ 3.7580, 11.4845,  6.8648, 10.4195],
        [ 4.2578, -0.1196, 10.1988, 11.1110]], grad_fn=<ThAddBackward>)


In [25]:
# autograd的计算结果与利用公式手动计算的结果一致
y.backward(t.ones(y.size()))
x.grad

tensor([[ 3.9426,  4.1530,  4.2083, 11.5991],
        [ 3.9502, 10.1025,  5.7802,  9.0812],
        [ 4.0762,  5.6249,  8.8697,  9.7444]])

In [26]:
gradmyf(x)

tensor([[ 3.9426,  4.1530,  4.2083, 11.5991],
        [ 3.9502, 10.1025,  5.7802,  9.0812],
        [ 4.0761,  5.6249,  8.8697,  9.7444]], grad_fn=<ThAddBackward>)

注意：grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以反向传播之前需把梯度清零。

## 神经网络工具箱 nn.Module
    
    用autograd可实现深度学习模型，但其抽象程度较低，如果用其来实现深度学习模型，则需要编写的代码量极大。在这种情况下，torch.nn应运而生，其是专门为深度学习而设计的模块。torch.nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络/层。下面先来看看如何用nn.Module实现自己的全连接层。全连接层，又名仿射层，输出y和输入x满足y=Wx+b，W和b是可学习的参数。


在nn.Module的子类中定义了forward函数，backward函数就会自动被实现(利用autograd)。在forward 函数中可使用任何tensor支持的函数，还可以使用if、for循环、print、log等Python语法，写法和标准的Python写法一致。
网络的可学习参数通过net.parameters()返回，net.named_parameters可同时返回可学习的参数及名称。

In [27]:
import torch as t
from torch import nn

In [28]:
class Linear(nn.Module): # 继承nn.Module
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__() # 等价于nn.Module.__init__(self)
        self.w = nn.Parameter(t.randn(in_features, out_features))
        self.b = nn.Parameter(t.randn(out_features))
    
    def forward(self, x):
        x = x.mm(self.w) # x.@(self.w)
        return x + self.b.expand_as(x)

In [32]:
linear_layer = Linear(5,2)
input_ = t.randn(4,5)
output = linear_layer(input_)
output

tensor([[-1.8883,  2.4866],
        [ 2.1829,  0.3224],
        [-1.7111, -1.4719],
        [ 0.1381,  4.3280]], grad_fn=<ThAddBackward>)

In [34]:
for name, parameter in linear_layer.named_parameters():
    print(name, parameter) # w and b 

w Parameter containing:
tensor([[ 0.8832,  0.2046],
        [-0.8670,  1.1250],
        [-0.3975,  0.5284],
        [ 0.4890,  1.3370],
        [ 1.3484, -0.0717]], requires_grad=True)
b Parameter containing:
tensor([-0.3180,  1.1186], requires_grad=True)


接下来，我们实现一个多层感知机。
其结构为：
![multi_perceptron](imgs/multi_perceptron.png)

两个全连接层，sigemoid函数作为activation function，图中没有画出

In [35]:
class Perceptron(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        nn.Module.__init__(self)
        self.layer1 = Linear(in_features, hidden_features) # 此处的Linear是前面自定义的全连接层
        self.layer2 = Linear(hidden_features, out_features)
    def forward(self,x):
        x = self.layer1(x)
        x = t.sigmoid(x)
        return self.layer2(x)

In [36]:
perceptron = Perceptron(3,4,1)
for name, param in perceptron.named_parameters():
    print(name, param.size())

layer1.w torch.Size([3, 4])
layer1.b torch.Size([4])
layer2.w torch.Size([4, 1])
layer2.b torch.Size([1])


PyTorch实现了神经网络中绝大多数的layer，这些layer都继承于nn.Module，封装了可学习参数parameter，并实现了forward函数，性能非常好。
阅读文档时应主要关注以下几点：

    构造函数的参数，如nn.Linear(in_features, out_features, bias)，需关注这三个参数的作用。
    属性、可学习参数和子module。如nn.Linear中有weight和bias两个可学习参数，不包含子module。
    输入输出的形状，如nn.linear的输入形状是(N, input_features)，输出为(N，output_features)，N是batch_size。

这些自定义layer对输入形状都有假设：输入的不是单个数据，而是一个batch。输入只有一个数据，则必须调用tensor.unsqueeze(0) 或 tensor[None]将数据伪装成batch_size=1的batch
 
这些层都继承于 nn.Module

```py
Conv：卷积层 实际使用中可分为一维(1D)、二维(2D)、三维（3D）等
Pool ：池化层 池化方式又分为平均池化（AvgPool）、最大值池化（MaxPool）、自适应池化（AdaptiveAvgPool）等
Linear：全连接层。
BatchNorm：批规范化层，分为1D、2D和3D。除了标准的BatchNorm之外，还有在风格迁移中常用到的InstanceNorm层。
Dropout：dropout层，用来防止过拟合，同样分为1D、2D和3D。 下面通过例子来说明它们的使用。
```

nn.Conv2d的功能是：对由多个输入平面组成的输入信号进行二维卷积，以最简单的例子进行说明：

输入信号的形式为(N,Cin,H,W)
(N,Cin​,H,W) ，NN表示batch size，CinCin​表示channel个数，HH，W

W分别表示特征图的高和宽。
![shape](imgs/conv2dshape.png)

参数说明：

    stride(步长)：控制cross-correlation的步长，可以设为1个int型数或者一个(int, int)型的tuple。

    padding(补0)：控制zero-padding的数目。

    dilation(扩张)：控制kernel点（卷积核点）的间距; 也被称为 "à trous"算法. 可以在此github地址查看:Dilated convolution animations

    groups(卷积核个数)：这个比较好理解，通常来说，卷积个数唯一，但是对某些情况，可以设置范围在1 —— in_channels中数目的卷积核：


池化层
对于输入信号的输入通道，提供2维最大池化（max pooling）操作

如果输入的大小是(N,C,H,W)，那么输出的大小是(N,C,H_out,W_out)和池化窗口大小(kH,kW)的关系是：
$$out(N_i, C_j,k)=max^{kH-1}{m=0}max^{kW-1}{m=0}
input(N_{i},C_j,stride[0]h+m,stride[1]w+n)$$ 
https://blog.csdn.net/g11d111/article/details/82665265

在以上的例子中，基本上都是将每一层的输出直接作为下一层的输入，这种网络称为前馈传播网络（feedforward neural network）。对于此类网络如果每次都写复杂的forward函数会有些麻烦，在此就有两种简化方式，ModuleList和Sequential。其中Sequential是一个特殊的module，它包含几个子Module，前向传播时会将输入一层接一层的传递下去。ModuleList也是一个特殊的module，可以包含几个子module，可以像用list一样使用它，但不能直接把输入传给ModuleList。下面举例说明。

In [37]:
# Sequential的三种写法
net1 = nn.Sequential()
net1.add_module('conv', nn.Conv2d(3, 3, 3))
net1.add_module('batchnorm', nn.BatchNorm2d(3))
net1.add_module('activation_layer', nn.ReLU())

net2 = nn.Sequential(
        nn.Conv2d(3, 3, 3),
        nn.BatchNorm2d(3),
        nn.ReLU()
        )

from collections import OrderedDict
net3= nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(3, 3, 3)),
          ('bn1', nn.BatchNorm2d(3)),
          ('relu1', nn.ReLU())
        ]))
print('net1:', net1)
print('net2:', net2)
print('net3:', net3)

net1: Sequential(
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation_layer): ReLU()
)
net2: Sequential(
  (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)
net3: Sequential(
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
)


根据上述的内容，我们可以根据lenet的网络结构，将lenet封装为lenet类：

```py
# 定义一个LeNet网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.features = nn.Sequential(
                    nn.Conv2d(3, 6, 5),
                    nn.ReLU(),
                    nn.MaxPool2d(2,2),
                    nn.Conv2d(6, 16, 5),
                    nn.ReLU(),
                    nn.MaxPool2d(2,2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 16 * 5 * 5)
        x = self.classifier(x)
        return x
``` 

nn中还有一个很常用的模块：nn.functional，nn中的大多数layer，在functional中都有一个与之相对应的函数。nn.functional中的函数和nn.Module的主要区别在于，用nn.Module实现的layers是一个特殊的类，都是由class layer(nn.Module)定义，会自动提取可学习的参数。而nn.functional中的函数更像是纯函数，由def function(input)定义。下面举例说明functional的使用，并指出二者的不同之处。


同时也可以写为：
       
```py
import torch.nn as nn
import torch.nn.functional as F
class lenet(nn.Module):
    def __init__(self):
        super(lenet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) 
        self.conv2 = nn.Conv2d(6, 16, 5)  
        self.fc1   = nn.Linear(16*5*5, 100)  
        self.fc2   = nn.Linear(100, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x): 
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) 
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) 
        x = x.view(x.size()[0], -1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)        
        return x
```

## 优化器
在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，例如随机梯度下降法(SGD)的更新策略如下：

weight = weight - learning_rate * gradient

手动实现如下：

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)# inplace 减法



PyTorch将深度学习中常用的优化方法全部封装在torch.optim中，其设计十分灵活，能够很方便的扩展成自定义的优化方法。

所有的优化方法都是继承基类optim.Optimizer，并实现了自己的优化步骤。下面就以最基本的优化方法——随机梯度下降法（SGD）举例说明。这里需重点掌握：

    优化方法的基本使用方法
    如何对模型的不同部分设置不同的学习率
    如何调整学习率
    
``` py
# SGD 就是随机梯度下降
opt_SGD         = torch.optim.SGD(net_SGD.parameters(), lr=LR)
# momentum 动量加速,在SGD函数里指定momentum的值即可
opt_Momentum    = torch.optim.SGD(net_Momentum.parameters(), lr=LR, momentum=0.8)
# RMSprop 指定参数alpha
opt_RMSprop     = torch.optim.RMSprop(net_RMSprop.parameters(), lr=LR, alpha=0.9)
# Adam 参数betas=(0.9, 0.99)
opt_Adam        = torch.optim.Adam(net_Adam.parameters(), lr=LR, betas=(0.9, 0.99))    
```  
 
pytorch中使用torch.optim优化神经网络以及优化器的选择 https://ptorch.com/news/54.html
pytorch调参https://blog.csdn.net/angnuan123/article/details/81604727



In [ ]:
import torch.optim as optim
#定义lenet结构和相关参数
class lenet(nn.Module):
    def __init__(self):
        super(lenet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) 
        self.conv2 = nn.Conv2d(6, 16, 5)  
        self.fc1   = nn.Linear(16*5*5, 100)  
        self.fc2   = nn.Linear(100, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x): 
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) 
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) 
        x = x.view(x.size()[0], -1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)        
        return x
    
    
#新建一个优化器，指定要调整的参数和学习率
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# 在训练过程中
# 先梯度清零(与net.zero_grad()效果一样)
optimizer.zero_grad() 

# 计算损失
output = lenet(input)
loss = criterion(output, target)

#反向传播
loss.backward()

#更新参数
optimizer.step()

## 数据处理

到目前为止，我们已经定义好了网络结构，网络所需要学习的参数也可以通过优化器不断的反向传播梯度可以优化得到。
 接下来，我们要把数据集的数据不断的地传送到网络中，让优化器一次一次地学习获取更优的参数，提高最终的结果。
 在解决深度学习问题的过程中，往往需要花费大量的精力去处理数据，包括图像、文本、语音或其它二进制数据等。数据的处理对训练神经网络来说十分重要，良好的数据处理不仅会加速模型训练，更会提高模型效果。考虑到这点，PyTorch提供了几个高效便捷的工具，以便使用者进行数据处理或增强等操作，同时可通过并行化加速数据加载。
 
 这可能是你代码量最高的一个部分。。。
 
 ### dataset
 在PyTorch中，数据加载可通过自定义的数据集对象。数据集对象被抽象为Dataset类，实现自定义的数据集需要继承Dataset，并实现两个Python魔法方法：

    __getitem__：返回一条数据，或一个样本。obj[index]等价于obj.__getitem__(index)
    __len__：返回样本的数量。len(obj)等价于obj.__len__()

思路就是设计一个类： 继承data.Dataset

分别实现以下三个方法：
def __init__(self, root) 告诉类你存放数据的目録初始化
def __getitem__(self, index) 通过index迭代，返回一条数据和label
def __len__(self) 返回样本的个数

实现了这几个方法

你设计的类因为继承data.Dataset 素以可以调用[0]来访问数据，并返回raw data 和 label

In [ ]:
import torch as t
from torch.utils import data
import os
from PIL import  Image
import numpy as np

class DogCat(data.Dataset):
    def __init__(self, root):
        imgs = os.listdir(root)
        # 所有图片的绝对路径
        # 这里不实际加载图片，只是指定路径，当调用__getitem__时才会真正读图片
        self.imgs = [os.path.join(root, img) for img in imgs]
        
    def __getitem__(self, index):
        img_path = self.imgs[index]
        # dog->1， cat->0
        label = 1 if 'dog' in img_path.split('/')[-1] else 0
        pil_img = Image.open(img_path)
        array = np.asarray(pil_img)
        data = t.from_numpy(array)
        return data, label
    
    def __len__(self):
        return len(self.imgs)

在这里介绍一个会经常使用到的Dataset——ImageFolder，它的实现和上述的DogCat很相似。ImageFolder假设所有的文件按文件夹保存，每个文件夹下存储同一个类别的图片，文件夹名为类名，其构造函数如下：

ImageFolder(root, transform=None, target_transform=None, loader=default_loader)

它主要有四个参数：

    root：在root指定的路径下寻找图片
    transform：对PIL Image进行的转换操作，transform的输入是使用loader读取图片的返回对象
    target_transform：对label的转换
    loader：给定路径后如何读取图片，默认读取为RGB格式的PIL Image对象

label是按照文件夹名顺序排序后存成字典，即{类名:类序号(从0开始)}，一般来说最好直接将文件夹命名为从0开始的数字，这样会和ImageFolder实际的label一致，如果不是这种命名规范，建议看看self.class_to_idx属性以了解label和文件夹名的映射关系。

In [ ]:
from torchvision.datasets import ImageFolder
dataset = ImageFolder('data/dogcat_2/')
# cat文件夹的图片对应label 0，dog对应1
dataset.class_to_idx
dataset.imgs

但这里返回的数据不适合实际使用，因其具有如下两方面问题：

    返回样本的形状不一，因每张图片的大小不一样，这对于需要取batch训练的神经网络来说很不友好
    返回样本的数值较大，未归一化至[-1, 1]
针对上述问题，PyTorch提供了torchvision^1。它是一个视觉工具包，提供了很多视觉图像处理的工具，其中transforms模块提供了对PIL Image对象和Tensor对象的常用操作。

对PIL Image的操作包括：

    Scale：调整图片尺寸，长宽比保持不变
    CenterCrop、RandomCrop、RandomResizedCrop： 裁剪图片
    Pad：填充
    ToTensor：将PIL Image对象转成Tensor，会自动将[0, 255]归一化至[0, 1]

对Tensor的操作包括：

    Normalize：标准化，即减均值，除以标准差
    ToPILImage：将Tensor转为PIL Image对象

如果要对图片进行多个操作，可通过Compose函数将这些操作拼接起来，类似于nn.Sequential。注意，这些操作定义后是以函数的形式存在，真正使用时需调用它的__call__方法，这点类似于nn.Module。例如要将图片调整为224×224，首先应构建这个操作trans = Resize((224, 224))，然后调用trans(img)。下面我们就用transforms的这些操作来优化上面实现的dataset。




```py
dataset = NewDogCat('data/dogcat_wrong/', transforms=transform)
```




torchvision已经预先实现了常用的Dataset，包括前面使用过的CIFAR-10，以及ImageNet、COCO、MNIST、LSUN等数据集，可通过诸如`torchvision.datasets.CIFAR10`来调用，具体使用方法请参看官方文档[^1]


Dataset只负责数据的抽象，一次调用__getitem__只返回一个样本。前面提到过，在训练神经网络时，最好是对一个batch的数据进行操作，同时还需要对数据进行shuffle和并行加速等。对此，PyTorch提供了DataLoader帮助我们实现这些功能。

pytorch 数据预处理： 样本-> dataset dataset -> dataloader

DataLoader的函数定义如下： DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, num_workers=0, collate_fn=default_collate, pin_memory=False, drop_last=False)

    dataset：加载的数据集(Dataset对象)
    batch_size：batch size
    shuffle:：是否将数据打乱
    sampler： 样本抽样，后续会详细介绍
    num_workers：使用多进程加载的进程数，0代表不使用多进程
    collate_fn： 如何将多个样本数据拼接成一个batch，一般使用默认的拼接方式即可
    pin_memory：是否将数据保存在pin memory区，pin memory中的数据转到GPU会快一些
    drop_last：dataset中的数据个数可能不是batch_size的整数倍，drop_last为True会将多出来不足一个batch的数据丢弃

[^1]: http://pytorch.org/docs/master/torchvision/datasets.html
